<a href="https://colab.research.google.com/github/HariAA/KaggleDataScience/blob/master/HS_Osmosis_Quora_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive


    100% |████████████████████████████████| 993kB 20.4MB/s 


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from keras.callbacks import Callback
import re
import gensim
from oauth2client.client import GoogleCredentials
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Embedding, LSTM, GRU ,Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Layer
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics import accuracy_score,confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import logging
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from google.colab import auth
# Any results you write to the current directory are saved as output.

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
listed = drive.ListFile({'q': "title contains '.bin'"}).GetList()
print(len(listed))
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

2
title GoogleNews-vectors-negative300.bin, id 1AZi1XiJMos56bN-fq9N9ASkOdMu1CPqT
title GoogleNews-vectors-negative300.bin.gz, id 0B7XkCwpI5KDYNlNUTTlSS21pQmM


In [0]:
train_file_id = '1fmaURsOvi4yMghZeDwEDqiXFBGet2VPy'
test_file_id = '1ibmBLfU_P9Hkg0Dh2HUTNS6y2mNKD6E3'
glove_file_id='1vF1l9mUNKc8fNlLTUy2AP4i5qvOeqkJW'
google_word2_vec_id='1AZi1XiJMos56bN-fq9N9ASkOdMu1CPqT'
downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile('train.csv')  
downloaded = drive.CreateFile({'id': test_file_id})
downloaded.GetContentFile('test.csv')  
downloaded = drive.CreateFile({'id': glove_file_id})
downloaded.GetContentFile('glove.txt')  
downloaded = drive.CreateFile({'id': google_word2_vec_id})
downloaded.GetContentFile('w2v.bin')  




In [5]:

train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
train_data.head(10)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,0
7,0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off...,0
8,00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, ...",0
9,00006e6928c5df60eacb,Is it just me or have you ever been in this ph...,0


In [0]:
def clean_data(questions):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\’)|(\‘)|(\”)|(\“)|(\…)|(\–)|(\→)|(\−)|(\∞)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    questions = [REPLACE_NO_SPACE.sub("", line.lower()) for line in questions]
    questions = [REPLACE_WITH_SPACE.sub(" ", line) for line in questions]
    questions = [re.sub(r'\d+', '', line) for line in questions]
    
    return questions

In [0]:
train_data['question_text'] = clean_data(train_data['question_text'])
test_data['question_text'] = clean_data(test_data['question_text'])

In [0]:
#x_train, x_cv, y_train, y_cv=train_test_split(train_data['question_text'],train_data['target'],test_size=0.2)
#x_train=train_data[train_data.target==[1]]['question_text']
x_train=train_data['question_text']
y_train=train_data['target']
x_test=test_data['question_text']

In [9]:
sentences = []
i=0
print("Parsing sentences from training set")
for question in x_train:
    if i % 100000 ==0: print(i)
    #print(gensim.utils.simple_preprocess(question))
    
    
    sentences.append(list(gensim.utils.simple_preprocess(question,min_len=1, max_len=50 )))
    #print(sentences[i])
    
    i+=1

Parsing sentences from training set
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000


In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
num_features=300
min_word_cnt=1
window=10
num_workers=4
#word_model=Word2Vec(sentences,workers=num_workers,size=num_features,min_count=min_word_cnt,window=window)

In [0]:
#word_vectors = KeyedVectors.load_word2vec_format('w2v.bin', binary=True)

In [13]:
print(x_train.shape)
print(x_train.append(x_test).shape)

(1306122,)
(1362492,)


In [14]:
tott=x_train.append(x_test)
tok = Tokenizer(lower=True)
tok.fit_on_texts(tott)
sequences_train = tok.texts_to_sequences(x_train)
sequences_test = tok.texts_to_sequences(x_test)
word_index=tok.word_index
num_words=len(word_index)+1
print(num_words)
pad_sequence_train=pad_sequences(sequences_train, maxlen=133)
pad_sequence_test=pad_sequences(sequences_test,maxlen=133)
print(pad_sequence_train.shape)
print(pad_sequence_test.shape)

198558
(1306122, 133)
(56370, 133)


In [15]:
embeddings_index = {}
f = open('glove.txt',encoding="utf8")
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

GloVe data loaded


In [17]:
embedding_matrix=np.zeros((num_words,300))
not_found=0
word_found=0
wc_found=0
not_found_words=[]
for word,i in word_index.items():
    try:
        #embedding_vector=word_vectors[word]
        embedding_vector=embeddings_index.get(word) 
        embedding_matrix[i]=embedding_vector
        word_found+=1
        if embedding_vector is None:
            #continue
            #embedding_matrix[i]=word_vectors[word]
            embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),300)
            #embedding_matrix[i][0]=25
            wc_found+=1
            #not_found_words.append((word,i))
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),300)
        not_found_words.append((word,i))
        #print(word)
        #print(i)
        #if i > 1000: break;
        not_found+=1
       # print('%s not found' %(word))
print('%s words found and %s words not found' %(word_found,wc_found))

198557 words found and 79233 words not found


In [0]:
embedding_layer=Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1], 
                            input_length=133,
                            weights=[embedding_matrix], 
                            trainable=False, 
                            name='embedding_layer')
model = Sequential()
model.add(embedding_layer)
model.add(GRU(units=128, dropout=0.2, recurrent_dropout=0.2))

#model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation='sigmoid'))

In [20]:
del embeddings_index

NameError: ignored

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])  # Compile the model
print(model.summary())  # Summarize the model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 133, 300)          59567400  
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               164736    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 59,732,265
Trainable params: 164,865
Non-trainable params: 59,567,400
_________________________________________________________________
None


In [0]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.v_f1 = []
        self.v_recall = []
        self.v_precision = []
 
    def on_epoch_end(self, epoch, logs={}):
        v_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        v_targ = self.validation_data[1]
        _v_f1 = f1_score(v_targ, v_predict)
        _v_recall = recall_score(v_targ, v_predict)
        _v_precision = precision_score(v_targ, v_predict)
        self.v_f1.append(_v_f1)
        self.v_recall.append(_v_recall)
        self.v_precision.append(_v_precision)
        print (' — v_f1: %f — v_precision: %f — v_recall %f' %(_v_f1, _v_precision, _v_recall))
        return
 
metrics = Metrics()


In [27]:
pad_sequence_t, pad_sequence_cv, y_t, y_cv=train_test_split(pad_sequence_train,y_train,test_size=0.2)
print(pad_sequence_t.shape)
print(pad_sequence_cv.shape)

(1044897, 133)
(261225, 133)


In [1]:
model.fit(pad_sequence_t, y_t, batch_size=128, epochs=5, verbose=1, validation_data=(pad_sequence_cv,y_cv), callbacks=[metrics])  # Fit the model
#loss, accuracy = model.evaluate(pad_sequence_t, labels, verbose=0)  # Evaluate the model
#print('Accuracy: %0.3f' % accuracy)

NameError: ignored

In [0]:
#word_index['which']
#word_vectors['which']
df_list=pd.DataFrame({'word':not_found_words, 'val':1})
#df_list['word']=df_list['word'].strip()
print(df_list.groupby(df_list['word']).sum().sort_values(['word'],ascending=False))

In [0]:
y_pred=model.predict_classes(pad_sequence_test)
submission=pd.DataFrame({'qid':test_data['qid'],'prediction':y_pred.ravel()},columns=['qid','prediction'])
submission.to_csv('submission.csv',index=False)